In [ ]:
!pip install numpy PyOpenGL pygame

import pygame
import numpy as np
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import math
import random
import time


# Constants
speed = 0.1  # Camera movement speed
sensitivity = 0.2  # Mouse sensitivity
jump_strength = 0.3  # Strength of the jump
gravity = 0.01  # Gravitational pull on the player
bounce_stop = 0.07
camera_height = 1.75  # Y position of the ground
map_size = 10
walls_height= 10
object_spawn_delay = 0.025
object_spawn_speed = 200.0
object_min_radius = 0.5
object_max_radius = 0.9
max_objects_count = 5
max_angle = 1.0
vertices = [
    [1, 0, -1],
    [1, 2, -1],
    [-1, 2, -1],
    [-1, 0, -1],
    [1, 0, 1],
    [1, 2, 1],
    [-1, 2, 1],
    [-1, 0, 1]
]
edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),  # Back face edges
    (4, 5), (5, 6), (6, 7), (7, 4),  # Front face edges
    (0, 4), (1, 5), (2, 6), (3, 7)   # Connecting edges
]

def get_rainbow(t):
    r = math.sin(t)
    g = math.sin(t + 0.33 * 2.0 * math.pi)
    b = math.sin(t + 0.66 * 2.0 * math.pi)
    return (
        int(255.0 * r * r),
        int(255.0 * g * g),
        int(255.0 * b * b)
    )

# First-person camera class
class Camera:
    def __init__(self):
        self.position = [0.0, 1.75, 5.0]  # Camera starting position
        self.yaw = 0.0  # Horizontal rotation (yaw)
        self.pitch = 0.0  # Vertical rotation (pitch)
        self.velocity_y = 0.0  # Vertical velocity for jumping/falling
        self.is_jumping = False  # Flag to check if the player is currently jumping

    # Update the camera's yaw and pitch based on mouse movement
    def rotate(self, mouse_dx, mouse_dy):
        self.yaw += mouse_dx * sensitivity
        self.pitch += mouse_dy * sensitivity
        self.pitch = max(-89.0, min(89.0, self.pitch))  # Limit pitch to prevent flipping

    # Move the camera forward, backward, left, right
    def move(self, direction):
        right = [
            math.cos(math.radians(self.yaw)),
            0.0,
            math.sin(math.radians(self.yaw))
        ]
        forward = [
            math.cos(math.radians(self.yaw + 90)),
            0.0,
            math.sin(math.radians(self.yaw + 90))
        ]

        if direction == 'FORWARD':
            self.position[0] -= forward[0] * speed
            self.position[2] -= forward[2] * speed
        elif direction == 'BACKWARD':
            self.position[0] += forward[0] * speed
            self.position[2] += forward[2] * speed
        elif direction == 'LEFT':
            self.position[0] -= right[0] * speed
            self.position[2] -= right[2] * speed
        elif direction == 'RIGHT':
            self.position[0] += right[0] * speed
            self.position[2] += right[2] * speed

    # Apply the camera transformation (position and rotation)
    def apply(self):
        # Apply pitch (look up/down) by rotating around the X-axis
        glRotatef(self.pitch, 1.0, 0.0, 0.0)
        # Apply yaw (look left/right) by rotating around the Y-axis
        glRotatef(self.yaw, 0.0, 1.0, 0.0)
        # Move the camera to the new position
        glTranslatef(-self.position[0], -self.position[1], -self.position[2])

    def update_jump(self):
        # If the player is in the air, apply gravity
        if self.position[1] > camera_height or self.is_jumping:
            self.velocity_y -= gravity  # Gravity pulls down

        # Update the camera's Y position based on velocity
        self.position[1] += self.velocity_y

        # Prevent the player from falling below the ground
        if self.position[1] < camera_height:
            self.position[1] = camera_height
            self.velocity_y = 0.0  # Reset velocity when on the ground
            self.is_jumping = False  # Reset the jump flag

    # Start jumping if the player is on the ground
    def jump(self):
        if self.position[1] == camera_height:
            self.velocity_y = jump_strength  # Set the initial jump velocity
            self.is_jumping = True  # Mark the player as jumping

def draw_cube():
    glBegin(GL_LINES)
    glColor3f(0.7, 0.7, 0.7)  # Light grey color for the grid
    for edge in edges:
        for vertex in edge:
            glVertex3fv(vertices[vertex])
    glEnd()

def draw_plane():
    # Draw a 3D plane as the ground
    glBegin(GL_QUADS)
    glColor3f(0.3, 0.6, 0.3)  # Green color for ground
    for x in range(-map_size, map_size):
        for y in range(-map_size, map_size):
            glVertex3f(x, 0, y)  # Ground level Z = 0
            glVertex3f(x + 1, 0, y)
            glVertex3f(x + 1, 0, y + 1)
            glVertex3f(x, 0, y + 1)
    glEnd()

    # Draw grid lines on top of the plane
    glColor3f(0.7, 0.7, 0.7)  # Light grey color for the grid
    glBegin(GL_LINES)
    
    # Draw vertical lines (X-axis)
    for x in range(-map_size, map_size + 1):
        glVertex3f(x, 0, -map_size)
        glVertex3f(x, 0, map_size)
    
    # Draw horizontal lines (Y-axis)
    for y in range(-map_size, map_size + 1):
        glVertex3f(-map_size, 0, y)
        glVertex3f(map_size, 0, y)
    
    glEnd()

def draw_walls():
    # Draw a 3D plane as the left
    glBegin(GL_QUADS)
    glColor3f(0.1, 0.0, 0.0)  # Green color for ground
    
    #front walls
    glVertex3f(-map_size, 0, map_size)  # Ground level Z = 0
    glVertex3f(-map_size, walls_height, map_size)
    glVertex3f(-map_size, walls_height, -map_size)
    glVertex3f(-map_size, 0, -map_size)
    glEnd()

    # Draw a 3D plane as the left
    glBegin(GL_QUADS)
    glColor3f(0.1, 0.0, 0.0)  # Green color for ground
    
    #front walls
    glVertex3f(map_size, 0, map_size)  # Ground level Z = 0
    glVertex3f(map_size, walls_height, map_size)
    glVertex3f(map_size, walls_height, -map_size)
    glVertex3f(map_size, 0, -map_size)
    glEnd()

    # Draw a 3D plane as the left
    glBegin(GL_QUADS)
    glColor3f(0.1, 0.0, 0.0)  # Green color for ground
    
    #front walls
    glVertex3f(map_size, 0, map_size)  # Ground level Z = 0
    glVertex3f(map_size, walls_height, map_size)
    glVertex3f(-map_size, walls_height, map_size)
    glVertex3f(-map_size, 0, map_size)
    glEnd()

    # Draw a 3D plane as the left
    glBegin(GL_QUADS)
    glColor3f(0.1, 0.0, 0.0)  # Green color for ground
    
    #front walls
    glVertex3f(map_size, 0, -map_size)  # Ground level Z = 0
    glVertex3f(map_size, walls_height, -map_size)
    glVertex3f(-map_size, walls_height, -map_size)
    glVertex3f(-map_size, 0, -map_size)
    glEnd()

color_map = {
    "red": (1.0, 0.0, 0.0),
    "green": (0.0, 1.0, 0.0),
    "blue": (0.0, 0.0, 1.0),
    "white": (1.0, 1.0, 1.0),
    "black": (0.0, 0.0, 0.0),
}

def set_color_by_name(color_name):
    color = color_map.get(color_name, (1.0, 1.0, 1.0))  # Default to white if color not found
    glColor3f(*color)

class VerletObject:
    def __init__(self, position, radius=10.0, color=(255, 255, 255)):
        self.position = np.array(position, dtype=np.float32)
        self.position_last = np.array(position, dtype=np.float32)
        self.acceleration = np.array([0.0, 0.0, 0.0], dtype=np.float32)
        self.radius = radius
        self.color = color

    def update(self, dt):
        # Compute how much we moved
        displacement = self.position - self.position_last
        # Update position
        self.position_last = self.position.copy()
        self.position = self.position + displacement + self.acceleration * (dt ** 2)
        # Reset acceleration
        self.acceleration = np.array([0.0, 0.0, 0.0], dtype=np.float32)

    def accelerate(self, a):
        self.acceleration += np.array(a, dtype=np.float32)

    def set_velocity(self, v, dt):
        self.position_last = self.position - np.array(v, dtype=np.float32) * dt

    def add_velocity(self, v, dt):
        self.position_last -= np.array(v, dtype=np.float32) * dt

    def get_velocity(self, dt):
        return (self.position - self.position_last) / dt

    def draw(self):
        glMatrixMode(GL_MODELVIEW)  # Switch to GL_MODELVIEW mode
        glEnable(GL_COLOR_MATERIAL)
        glPushMatrix()  # Save the current matrix state
        glTranslatef(self.position[0], self.position[1], self.position[2])  # Move to the sphere's position
        quadric = gluNewQuadric()  # Create a new quadric object
        set_color_by_name(self.color)
        gluSphere(quadric, self.radius, 30, 30)  # Draw the sphere (radius, slices, stacks)
        gluDeleteQuadric(quadric)  # Clean up the quadric object
        glPopMatrix()  # Restore the matrix state

class Solver:
    def __init__(self):
        self.objects = []
        self.gravity = np.array([0.0, -10.0, 0], dtype=np.float32)
        self.constraint_min = np.array([-10.0, 0.0, -10.0], dtype=np.float32)
        self.constraint_max = np.array([10.0, 10.0, 10.0], dtype=np.float32)
        self.sub_steps = 1
        self.frame_dt = 0.0
        self.time = 0.0
        self.bounce_retention = 0.9  # Retention factor for bouncing

    def add_object(self, position, radius, color):
        obj = VerletObject(position, radius, color)
        self.objects.append(obj)
        return obj

    def update(self):
        self.time += self.frame_dt
        step_dt = self.get_step_dt()
        for _ in range(self.sub_steps):
            self.apply_gravity()
            self.check_collisions(step_dt)
            self.apply_constraint()
            self.update_objects(step_dt)

    def set_simulation_update_rate(self, rate):
        self.frame_dt = 1.0 / rate

    def set_constraint(self, min_position, max_position):
        self.constraint_min = np.array(min_position, dtype=np.float32)
        self.constraint_max = np.array(max_position, dtype=np.float32)

    def set_sub_steps_count(self, sub_steps):
        self.sub_steps = sub_steps

    def set_object_velocity(self, obj, v):
        obj.set_velocity(v, self.get_step_dt())

    def get_objects(self):
        return self.objects

    def get_constraint(self):
        return (*self.constraint_min, *self.constraint_max)

    def get_objects_count(self):
        return len(self.objects)

    def get_time(self):
        return self.time

    def get_step_dt(self):
        return self.frame_dt / self.sub_steps

    def apply_gravity(self):
        for obj in self.objects:
            obj.accelerate(self.gravity)

    def check_collisions(self, dt):
        response_coef = 0.75
        objects_count = len(self.objects)
        for i in range(objects_count):
            object_1 = self.objects[i]
            for k in range(i + 1, objects_count):
                object_2 = self.objects[k]
                v = object_1.position - object_2.position
                dist2 = np.dot(v, v)
                min_dist = object_1.radius + object_2.radius
                if dist2 < min_dist * min_dist:
                    dist = np.sqrt(dist2)
                    n = v / dist
                    mass_ratio_1 = object_1.radius / (object_1.radius + object_2.radius)
                    mass_ratio_2 = object_2.radius / (object_1.radius + object_2.radius)
                    delta = 0.5 * response_coef * (dist - min_dist)
                    object_1.position -= n * (mass_ratio_2 * delta)
                    object_2.position += n * (mass_ratio_1 * delta)

    def apply_constraint(self):
        for obj in self.objects:
            # Apply constraints on x and z within the bounds and y between 0 and height
            if obj.position[0] < self.constraint_min[0] or obj.position[0] > self.constraint_max[0]:
                obj.position[0] = np.clip(obj.position[0], self.constraint_min[0], self.constraint_max[0])
                obj.position_last[0] = obj.position[0] + (obj.position_last[0] - obj.position[0]) * -self.bounce_retention

            if obj.position[1] < 0 or obj.position[1] > self.constraint_max[1]:
                obj.position[1] = np.clip(obj.position[1], 0, self.constraint_max[1])
                obj.position_last[1] = obj.position[1] + (obj.position_last[1] - obj.position[1]) * -self.bounce_retention

            if obj.position[2] < self.constraint_min[2] or obj.position[2] > self.constraint_max[2]:
                obj.position[2] = np.clip(obj.position[2], self.constraint_min[2], self.constraint_max[2])
                obj.position_last[2] = obj.position[2] + (obj.position_last[2] - obj.position[2]) * -self.bounce_retention

    def update_objects(self, dt):
        for obj in self.objects:
            obj.update(dt)
    
def main():
    pygame.init()
    display = (800, 600)
    pygame.display.set_mode(display, DOUBLEBUF | OPENGL)
    clock = pygame.time.Clock()

    # glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45, (display[0] / display[1]), 0.1, 100.0)  # Perspective projection

    # Switch to ModelView Matrix (for object transformations)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

    camera = Camera()
    
    pygame.mouse.set_visible(False)  # Hide the mouse for first-person control
    pygame.mouse.set_pos(display[0] // 2, display[1] // 2)  # Center mouse
    pygame.event.set_grab(True)  # Capture mouse input for smooth camera movement

    solver = Solver()
    solver.set_simulation_update_rate(60)
    solver.set_sub_steps_count(4)
    objects = []
    spawn_timer = time.time()

    # Main loop
    while True:
        dt = clock.tick(60) / 1000.0
        solver.frame_dt = dt

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
        
        keys = pygame.key.get_pressed()
        if keys[K_w]:
            camera.move('FORWARD')
        if keys[K_s]:
            camera.move('BACKWARD')
        if keys[K_a]:
            camera.move('LEFT')
        if keys[K_d]:
            camera.move('RIGHT')

        if keys[K_SPACE]:
            camera.jump()
        if keys[K_r]:
            objects = []

        # Get mouse movement for camera rotation
        mouse_dx, mouse_dy = pygame.mouse.get_rel()
        camera.rotate(mouse_dx, mouse_dy)
        camera.update_jump()

        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

        # Apply the camera transformations (first-person view)
        glLoadIdentity()
        camera.apply()

        if len(objects) < max_objects_count and (time.time() - spawn_timer) >= object_spawn_delay:
            spawn_timer = time.time()
            radius = random.uniform(object_min_radius, object_max_radius)
            position = (0, 0, 0)
            t = pygame.time.get_ticks() / 1000  # time in seconds
            obj = solver.add_object(position, radius, get_rainbow(t))
            angle = max_angle * math.sin(t) + math.pi * 0.5
            obj.set_velocity(object_spawn_speed * pygame.math.Vector3(math.cos(angle), 270 ,math.sin(angle)), dt)
            objects.append(obj)

        solver.update()
        draw_plane()
        draw_walls()
        draw_cube()
        for obj in solver.get_objects():
            obj.draw()
        pygame.display.flip()
        pygame.time.wait(10)

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: video system not initialized

: 

In [7]:
import pygame
import numpy as np

class VerletObject:
    def __init__(self, position, radius=10.0, color=(255, 255, 255)):
        self.position = np.array(position, dtype=np.float32)
        self.position_last = np.array(position, dtype=np.float32)
        self.acceleration = np.array([0.0, 0.0], dtype=np.float32)
        self.radius = radius
        self.color = color

    def update(self, dt):
        # Compute how much we moved
        displacement = self.position - self.position_last
        # Update position
        self.position_last = self.position.copy()
        self.position = self.position + displacement + self.acceleration * (dt ** 2)
        # Reset acceleration
        self.acceleration = np.array([0.0, 0.0], dtype=np.float32)

    def accelerate(self, a):
        self.acceleration += np.array(a, dtype=np.float32)

    def set_velocity(self, v, dt):
        self.position_last = self.position - np.array(v, dtype=np.float32) * dt

    def add_velocity(self, v, dt):
        self.position_last -= np.array(v, dtype=np.float32) * dt

    def get_velocity(self, dt):
        return (self.position - self.position_last) / dt

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, self.position.astype(int), int(self.radius))

# Example Pygame usage
def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    clock = pygame.time.Clock()

    verlet_object = VerletObject(position=[400, 300], radius=15, color=(255, 0, 0))

    running = True
    while running:
        dt = clock.tick(60) / 1000.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Apply gravity as an example acceleration
        verlet_object.accelerate([0, 98.1])

        # Update the verlet object
        verlet_object.update(dt)

        # Draw everything
        screen.fill((0, 0, 0))
        verlet_object.draw(screen)
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()
